In [1]:
import numpy as np
from sklearn.datasets import fetch_openml # We are using it only once to load mnist dataset

# Load MNIST data
mnist = fetch_openml('mnist_784', as_frame=False)
X, y = mnist.data, mnist.target
X = X / 255.0  # Normalize pixel values
y = y.astype(int)  # Convert labels to integers

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
def initialize():
    # Initializing weights and biases for 3 layers
    weights1 = np.random.randn(784, 104) * np.sqrt(2. / 784)
    bias1 = np.zeros((1, 104))

    weights2 = np.random.randn(104, 64) * np.sqrt(2. / 104)
    bias2 = np.zeros((1, 64))

    weights3 = np.random.randn(64, 10) * np.sqrt(2. / 64)
    bias3 = np.zeros((1, 10))

    # Initializing velocity terms for momentum
    v_w1 = np.zeros_like(weights1)
    v_b1 = np.zeros_like(bias1)
    v_w2 = np.zeros_like(weights2)
    v_b2 = np.zeros_like(bias2)
    v_w3 = np.zeros_like(weights3)
    v_b3 = np.zeros_like(bias3)

    return weights1, bias1, weights2, bias2, weights3, bias3, v_w1, v_b1, v_w2, v_b2, v_w3, v_b3

In [3]:
def relu(z):
    return np.maximum(0, z)

def relu_deriv(z):
    return (z > 0).astype(float)

def softmax(z):
    exp = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtracting maximum to prevent overflow
    return exp / np.sum(exp, axis=1, keepdims=True)

In [4]:
def forward(X, w1, b1, w2, b2, w3, b3):
    z1 = np.dot(X, w1) + b1
    a1 = relu(z1)

    z2 = np.dot(a1, w2) + b2
    a2 = relu(z2)

    z3 = np.dot(a2, w3) + b3
    a3 = softmax(z3)

    return z1, a1, z2, a2, z3, a3

In [5]:
def one_hot_encode(y, num_classes):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y] = 1
    return one_hot

In [6]:
def back_prop(X, y, z1, a1, z2, a2, z3, a3, w1, w2, w3, lambda_reg):
    m = X.shape[0]
    y = one_hot_encode(y, 10)

    dz3 = a3 - y
    dw3 = np.dot(a2.T, dz3) / m + (lambda_reg / m) * w3  # L2 regularization
    db3 = np.sum(dz3, axis=0, keepdims=True) / m

    dz2 = np.dot(dz3, w3.T) * relu_deriv(z2)
    dw2 = np.dot(a1.T, dz2) / m + (lambda_reg / m) * w2  # L2 regularization
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    dz1 = np.dot(dz2, w2.T) * relu_deriv(z1)
    dw1 = np.dot(X.T, dz1) / m + (lambda_reg / m) * w1  # L2 regularization
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    return dw1, db1, dw2, db2, dw3, db3

In [7]:
def update_params_nag(w1, b1, w2, b2, w3, b3, dw1, db1, dw2, db2, dw3, db3,
                      v_w1, v_b1, v_w2, v_b2, v_w3, v_b3, alpha, beta):
    # Lookahead step
    lookahead_w1 = w1 - beta * v_w1
    lookahead_b1 = b1 - beta * v_b1
    lookahead_w2 = w2 - beta * v_w2
    lookahead_b2 = b2 - beta * v_b2
    lookahead_w3 = w3 - beta * v_w3
    lookahead_b3 = b3 - beta * v_b3

    # Compute gradients at the lookahead positions
    z1_lookahead, a1_lookahead, z2_lookahead, a2_lookahead, z3_lookahead, a3_lookahead = forward(
        X_train, lookahead_w1, lookahead_b1, lookahead_w2, lookahead_b2, lookahead_w3, lookahead_b3
    )
    dw1_lookahead, db1_lookahead, dw2_lookahead, db2_lookahead, dw3_lookahead, db3_lookahead = back_prop(
        X_train, y_train, z1_lookahead, a1_lookahead, z2_lookahead, a2_lookahead, z3_lookahead, a3_lookahead,
        lookahead_w1, lookahead_w2, lookahead_w3, lambda_reg=0.01
    )

    # Update velocities
    v_w1 = beta * v_w1 + (1 - beta) * dw1_lookahead
    v_b1 = beta * v_b1 + (1 - beta) * db1_lookahead
    v_w2 = beta * v_w2 + (1 - beta) * dw2_lookahead
    v_b2 = beta * v_b2 + (1 - beta) * db2_lookahead
    v_w3 = beta * v_w3 + (1 - beta) * dw3_lookahead
    v_b3 = beta * v_b3 + (1 - beta) * db3_lookahead

    # Update parameters with momentum
    w1 -= alpha * v_w1
    b1 -= alpha * v_b1
    w2 -= alpha * v_w2
    b2 -= alpha * v_b2
    w3 -= alpha * v_w3
    b3 -= alpha * v_b3

    return w1, b1, w2, b2, w3, b3, v_w1, v_b1, v_w2, v_b2, v_w3, v_b3

In [8]:
def get_predictions(A3):
    return np.argmax(A3, axis=1)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [11]:
def train(X, y, input_size, hidden_size1, hidden_size2, output_size, epochs, learning_rate, lambda_reg, beta):
    weights1, bias1, weights2, bias2, weights3, bias3, v_w1, v_b1, v_w2, v_b2, v_w3, v_b3 = initialize()

    for epoch in range(epochs):
        z1, a1, z2, a2, z3, a3 = forward(X, weights1, bias1, weights2, bias2, weights3, bias3)
        dw1, db1, dw2, db2, dw3, db3 = back_prop(X, y, z1, a1, z2, a2, z3, a3, weights1, weights2, weights3, lambda_reg)
        weights1, bias1, weights2, bias2, weights3, bias3, v_w1, v_b1, v_w2, v_b2, v_w3, v_b3 = update_params_nag(
            weights1, bias1, weights2, bias2, weights3, bias3,
            dw1, db1, dw2, db2, dw3, db3,
            v_w1, v_b1, v_w2, v_b2, v_w3, v_b3,
            learning_rate, beta
        )

        if epoch % 10 == 0:
            loss = -np.mean(np.sum(one_hot_encode(y, 10) * np.log(a3 + 1e-8), axis=1))  # Numerical stability with log
            reg_loss = (lambda_reg / (2 * X.shape[0])) * (np.sum(np.square(weights1)) + np.sum(np.square(weights2)) + np.sum(np.square(weights3)))  # L2 regularization term
            total_loss = loss + reg_loss
            print(f'Epoch {epoch}, Loss: {total_loss}')

    return weights1, bias1, weights2, bias2, weights3, bias3

In [10]:
def predict(X, weights1, bias1, weights2, bias2, weights3, bias3):
    _, _, _, _, _, a3 = forward(X, weights1, bias1, weights2, bias2, weights3, bias3)
    return np.argmax(a3, axis=1)

In [12]:
# Train the model
weights1, bias1, weights2, bias2, weights3, bias3 = train(X_train, y_train, 784, 104, 64, 10, epochs=1000, learning_rate=0.01, lambda_reg=0.01, beta=0.9)

Epoch 0, Loss: 2.358157852299036
Epoch 10, Loss: 2.3112128691328135
Epoch 20, Loss: 2.2395714135505917
Epoch 30, Loss: 2.159444779000644
Epoch 40, Loss: 2.075731679716933
Epoch 50, Loss: 1.9907878414258813
Epoch 60, Loss: 1.9061527146942026
Epoch 70, Loss: 1.8230396244272462
Epoch 80, Loss: 1.74234808419861
Epoch 90, Loss: 1.6647069316455296
Epoch 100, Loss: 1.5905549397776477
Epoch 110, Loss: 1.5201652438018858
Epoch 120, Loss: 1.4536548047304603
Epoch 130, Loss: 1.3910079738028926
Epoch 140, Loss: 1.3321933283300609
Epoch 150, Loss: 1.2771140641776348
Epoch 160, Loss: 1.2256000598862018
Epoch 170, Loss: 1.1774814027296319
Epoch 180, Loss: 1.132587729516382
Epoch 190, Loss: 1.0907200010148994
Epoch 200, Loss: 1.0516771754777827
Epoch 210, Loss: 1.0152795917315034
Epoch 220, Loss: 0.9813379625645288
Epoch 230, Loss: 0.949678060943342
Epoch 240, Loss: 0.9201361517884574
Epoch 250, Loss: 0.8925548117677374
Epoch 260, Loss: 0.8667822647498076
Epoch 270, Loss: 0.8426813329817554
Epoch 280,

In [13]:
# Evaluate the model
predictions = predict(X_test, weights1, bias1, weights2, bias2, weights3, bias3)
accuracy = get_accuracy(predictions, y_test)
print(f'Test Accuracy: {accuracy * 100}%')

Test Accuracy: 90.06%
